# Feature Selection

## A new Enron Feature Quiz
Using `poi_flag_email.py` and `studentMain.py` but standing by, since `reader.py` was inaccessible

In [ ]:
import sys
import reader
sys.path.append('../final_project/')
import poi_email_addresses as poi_emails
import os
import zipfile

def getToFromStrings(f):
    '''
    The imported reader.py file contains functions that we've created to help
    parse e-mails from the corpus. .getAddresses() reads in the opening lines
    of an e-mail to find the To: From: and CC: strings, while the
    .parseAddresses() line takes each string and extracts the e-mail addresses
    as a list.
    '''
    f.seek(0)
    to_string, from_string, cc_string   = reader.getAddresses(f)
    to_emails   = reader.parseAddresses( to_string )
    from_emails = reader.parseAddresses( from_string )
    cc_emails   = reader.parseAddresses( cc_string )

    return to_emails, from_emails, cc_emails


### POI flag an email

def poiFlagEmail(f):
    """ given an email file f,
        return a trio of booleans for whether that email is
        to, from, or cc'ing a poi """

    to_emails, from_emails, cc_emails = getToFromStrings(f)

    ### poi_emails.poiEmails() returns a list of all POIs' email addresses.
    poi_email_list = poi_emails.poiEmails()

    to_poi = False
    from_poi = False
    cc_poi   = False

    ### to_poi and cc_poi are related functions, which flag whether
    ### the email under inspection is addressed to a POI, or if a POI is in cc
    ### you don't have to change this code at all

    ### there can be many "to" emails, but only one "from", so the
    ### "to" processing needs to be a little more complicated
    if to_emails:
        ctr = 0
        while not to_poi and ctr < len(to_emails):
            if to_emails[ctr] in poi_email_list:
                to_poi = True
            ctr += 1
    if cc_emails:
        ctr = 0
        while not to_poi and ctr < len(cc_emails):
            if cc_emails[ctr] in poi_email_list:
                cc_poi = True
            ctr += 1


    #################################
    ######## your code below ########
    ### set from_poi to True if #####
    ### the email is from a POI #####
    #################################

    
    
    

    #################################
    return to_poi, from_poi, cc_poi

## Visualizing your new feature
Using `studentCode.py`

In [3]:
import pickle
from get_data import getData

In [6]:
def computeFraction( poi_messages, all_messages ):
    """ given a number messages to/from POI (numerator) 
        and number of all messages to/from a person (denominator),
        return the fraction of messages to/from that person
        that are from/to a POI
   """


    ### you fill in this code, so that it returns either
    ###     the fraction of all messages to this person that come from POIs
    ###     or
    ###     the fraction of all messages from this person that are sent to POIs
    ### the same code can be used to compute either quantity

    ### beware of "NaN" when there is no known email address (and so
    ### no filled email features), and integer division!
    ### in case of poi_messages or all_messages having "NaN" value, return 0.
    fraction = 0.

    if poi_messages == 'NaN' or all_messages == 'NaN':
        return fraction
    
    fraction = 1.0 * poi_messages / all_messages


    return fraction


data_dict = getData() 

submit_dict = {}
for name in data_dict:

    data_point = data_dict[name]

#     print
    from_poi_to_this_person = data_point["from_poi_to_this_person"]
    to_messages = data_point["to_messages"]
    fraction_from_poi = computeFraction( from_poi_to_this_person, to_messages )
#     print fraction_from_poi
    data_point["fraction_from_poi"] = fraction_from_poi


    from_this_person_to_poi = data_point["from_this_person_to_poi"]
    from_messages = data_point["from_messages"]
    fraction_to_poi = computeFraction( from_this_person_to_poi, from_messages )
#     print fraction_to_poi
    submit_dict[name]={"from_poi_to_this_person":fraction_from_poi,
                       "from_this_person_to_poi":fraction_to_poi}
    data_point["fraction_to_poi"] = fraction_to_poi
    
    
#####################
submit_dict

{'ALLEN PHILLIP K': {'from_poi_to_this_person': 0.016195727084769126,
  'from_this_person_to_poi': 0.029612756264236904},
 'BADUM JAMES P': {'from_poi_to_this_person': 0.0,
  'from_this_person_to_poi': 0.0},
 'BANNANTINE JAMES M': {'from_poi_to_this_person': 0.06890459363957598,
  'from_this_person_to_poi': 0.0},
 'BAXTER JOHN C': {'from_poi_to_this_person': 0.0,
  'from_this_person_to_poi': 0.0},
 'BAY FRANKLIN R': {'from_poi_to_this_person': 0.0,
  'from_this_person_to_poi': 0.0},
 'BAZELIDES PHILIP J': {'from_poi_to_this_person': 0.0,
  'from_this_person_to_poi': 0.0},
 'BECK SALLY W': {'from_poi_to_this_person': 0.01968557758031442,
  'from_this_person_to_poi': 0.08887865530739121},
 'BELDEN TIMOTHY N': {'from_poi_to_this_person': 0.028532098610937303,
  'from_this_person_to_poi': 0.2231404958677686},
 'BELFER ROBERT': {'from_poi_to_this_person': 0.0,
  'from_this_person_to_poi': 0.0},
 'BERBERIAN DAVID': {'from_poi_to_this_person': 0.0,
  'from_this_person_to_poi': 0.0},
 'BERGSIE

## Feature Selection Mini-Project
Running a decision tree on the emails data

In [7]:
#imports 
import pickle
import numpy
numpy.random.seed(42)

In [8]:
### The words (features) and authors (labels), already largely processed.
### These files should have been created from the previous (Lesson 10)
### mini-project.
words_file = "../text_learning/your_word_data.pkl" 
authors_file = "../text_learning/your_email_authors.pkl"
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )

In [9]:
### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()

/home/rad/ml_week_py2/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [12]:
def classify(features_train, labels_train, min_samples_split = 40):
    clf = DecisionTreeClassifier(min_samples_split=min_samples_split)
    clf.fit(features_train, labels_train)
    return clf

In [14]:
clf = classify(features_train, labels_train, min_samples_split=2)
print "Test accuracy is", accuracy_score(clf.predict(features_test), labels_test)

Test accuracy is 0.947667804323


In [17]:
for i, imp in enumerate(clf.feature_importances_):
    if imp > 0.2:
        print "feature #", i, "has an importance of", imp

feature # 33614 has an importance of 0.764705882353


In [21]:
vectorizer.get_feature_names()[33614]

u'sshacklensf'

In [22]:
#Removing 'sshacklensf' from word_data
for i in range(len(word_data)):
    word_data[i] = word_data[i].replace('sshacklensf','')

In [29]:
#Recreating data
def get_data(word_data):
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    return features_train, features_test, labels_train, labels_test    

In [ ]:

features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()

In [25]:
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]
clf = classify(features_train, labels_train, min_samples_split=2)
print "Test accuracy is", accuracy_score(clf.predict(features_test), labels_test)

Test accuracy is 0.969283276451


In [26]:
for i, imp in enumerate(clf.feature_importances_):
    if imp > 0.2:
        print "feature #", i, "has an importance of", imp

feature # 14343 has an importance of 0.666666666667


In [ ]:
print "new outlier word is", vectorizer.get_feature_names()[14343]

In [ ]:
#Rerunning again by taking out 'cgermannsf'
for i in range(len(word_data)):
    word_data[i] = word_data[i].replace('cgermannsf','')

features_train, features_test, labels_train, labels_test = get_data(word_data)

features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]
clf = classify(features_train, labels_train, min_samples_split=2)

In [33]:
print "Test accuracy is", accuracy_score(clf.predict(features_test), labels_test)
for i, imp in enumerate(clf.feature_importances_):
    if imp > 0.2:
        print "feature '"+ vectorizer.get_feature_names()[i] +"' has an importance of", imp

Test accuracy is 0.813424345848
feature 'houectect' has an importance of 0.363636363636
